In [1]:
"""
Description:
    Load an analsyis grid, sky matrix and surfaces json, and prepare and run
    the radiance simulation for Daylight Factor and Daylight Autonomy.
    To run enter the command:
    python 02_RunRadianceJSON.py <path to analysis grid json> <path to surfaces JSON file> <path to sky matrix JSON file>
Arguments:
    path [string]: JSON config file (and referenced IDF within that config file)
Returns:
    Radiance simulation recipe/s [file objects]: Radiance simulation input/s

Annotations:
"""

# Load the necesasary packages
import json
import os
import sys
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')
from honeybee.schedule import Schedule
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as GridBasedDF
from honeybee.radiance.recipe.annual.gridbased import GridBased as GridBasedAnnual
from honeybee.futil import bat_to_sh


def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)

# Gather the paths to the provided resources
ANALYSIS_GRID_PATH = "HoneybeeRecipeJSONs/AnalysisGrids/zone1.json"
SURFACES_PATH = "HoneybeeRecipeJSONs/surfaces.json"
SKY_MTX_PATH = "HoneybeeRecipeJSONs/sky_mtx.json"

# Load the data from the constituent parts
SKY_MTX = load_json(SKY_MTX_PATH)
print("\nSky matrix loaded from {0:}\n".format(SKY_MTX_PATH))

SURFACES = load_json(SURFACES_PATH)
print("Surfaces (context geometry) loaded from {0:}\n".format(SURFACES_PATH))

ANALYSIS_GRID = load_json(ANALYSIS_GRID_PATH)
print("Analysis grid loaded from {0:}\n".format(ANALYSIS_GRID_PATH))

# Define configurations for the Annual and DF simulations
ANNUAL_CONFIG = {
    "type": "gridbased",
    "id": "annual",
    "simulation_type": 0,
    "rad_parameters": {
        "gridbased_parameters":
        "-I -aa 0.25 -dj 0.0 -ds 0.5 -dr 3 -ss 0.0 -dp 64 -ad 5000 -st 0.85 -lw 2e-06 -as 128 -dc 0.25 -dt 0.5 -ab 3 -c 1 -lr 4 -ar 16"
    }
}

DF_CONFIG = {
    "rad_parameters": {
        "gridbased_parameters":
        "-aa 0.25 -dj 0.0 -ds 0.5 -ss 0.0 -dp 64 -ad 512 -st 0.85 -lw 0.05 -as 128 -dc 0.25 -dt 0.5 -ab 2 -lr 4 -dr 3 -ar 16"
    },
    "type": "gridbased",
    "id": "daylight_factor"
}

# Create recipes from JSON origin string
DF_RECIPE_JSON = {k: v for d in [DF_CONFIG, ANALYSIS_GRID, SURFACES] for k, v in d.items()}
ANNUAL_RECIPE_JSON = {k: v for d in [ANNUAL_CONFIG, ANALYSIS_GRID, SURFACES, SKY_MTX] for k, v in d.items()}

# Create recipe from JSON origin string
DF_RECIPE = GridBasedDF.from_json(DF_RECIPE_JSON)
print("Daylight Factor recipe prepared\n")

ANNUAL_RECIPE = GridBasedAnnual.from_json(ANNUAL_RECIPE_JSON)
print("Annual recipe prepared\n")

# Generate bat and sh files
DF_BAT_FILE = DF_RECIPE.write("HoneybeeRecipeJSONs", str(DF_RECIPE.analysis_grids[0].name))
DF_SHELL_FILE = bat_to_sh(DF_BAT_FILE)
print("Daylight Factor recipe converted to Radiance case\n")

ANNUAL_BAT_FILE = ANNUAL_RECIPE.write("test")
ANNUAL_SHELL_FILE = bat_to_sh(ANNUAL_BAT_FILE)
print("Annual recipe converted to Radiance case\n")


print("Running Daylight Factor simulation...\n")
if os.name == 'nt':
    DF_RECIPE.run(DF_BAT_FILE)
else:
    DF_RECIPE.run(DF_SHELL_FILE)
print("Daylight Factor simulation complete\n")

print("Running Annual simulation...\n")
if os.name == 'nt':
    ANNUAL_RECIPE.run(ANNUAL_BAT_FILE)
else:
    ANNUAL_RECIPE.run(ANNUAL_SHELL_FILE)
print("Annual simulation complete\n")

# Post process the results and prepare a single results file for the zone analysed

# Get the X, Y, Z coordinates for the analysis points
x, y, z = list(zip(*[i["location"] for i in DF_RECIPE.results()[0].to_json()["analysis_points"]]))

# Get the Daylight Factor values corresponding with the analysis points
df = [point["values"][0][0][6324][0] for point in DF_RECIPE.results()[0].to_json()["analysis_points"]]

# Get the annual metrics from the annual results
OCCUPANCY_SCHEDULE = Schedule.from_workday_hours(occ_hours=(8, 17), off_hours=(12, 13), weekend=(6, 7), default_value=1)


print(ANNUAL_RECIPE.results()[0])

Path to radiance is set to: c:/radiance

Sky matrix loaded from HoneybeeRecipeJSONs/sky_mtx.json

Surfaces (context geometry) loaded from HoneybeeRecipeJSONs/surfaces.json

Analysis grid loaded from HoneybeeRecipeJSONs/AnalysisGrids/zone1.json

Updated value for ambient_accuracy to 0.25
Updated value for direct_jitter to 0.0
Updated value for direct_sampling to 0.5
Updated value for specular_sampling to 0.0
Updated value for direct_presamp_density to 64
Updated value for ambient_divisions to 512
Updated value for specular_threshold to 0.85
Updated value for limit_weight to 0.05
Updated value for ambient_supersamples to 128
Updated value for direct_certainty to 0.25
Updated value for direct_threshold to 0.5
Updated value for ambient_bounces to 2
Updated value for limit_reflections to 4
Updated value for direct_sec_relays to 3
Updated value for ambient_resolution to 16
Found 160 opaque surfaces.
Found 20 fenestration surfaces.
Found 0 window-groups.
Daylight Factor recipe prepared

Updat

In [3]:
ANNUAL_RECIPE.results()[0]

ANNUAL_RECIPE.result_files

OCCUPANCY_SCHEDULE = Schedule.from_workday_hours(occ_hours=(8, 17), off_hours=(12, 13), weekend=(6, 7), default_value=1)
grid = ANNUAL_RECIPE.results()[0]
da, cda, udi, udiless, udimore = grid.annual_metrics(300, (100, 2000), None, OCCUPANCY_SCHEDULE)



Unloading the current values from the analysis grids.

Adding test\untitled\gridbased_annual\result/scene..default.ill and test\untitled\gridbased_annual\result\sun..scene..default.ill to result files for zone1

XXXXXXXXXXtest\untitled\gridbased_annual\result/scene..default.ill
XXXXXXXXXXtest\untitled\gridbased_annual\result\sun..scene..default.ill
Unloading the current values from the analysis grids.

Adding test\untitled\gridbased_annual\result/scene..default.ill and test\untitled\gridbased_annual\result\sun..scene..default.ill to result files for zone1

XXXXXXXXXXtest\untitled\gridbased_annual\result/scene..default.ill
XXXXXXXXXXtest\untitled\gridbased_annual\result\sun..scene..default.ill
Loading the results from result files.


In [7]:
udimore

[57.77272727272727, 46.22727272727273, 37.0, 16.09090909090909]

In [ ]:
# Generate and run a Radiance case from some recipe inputs
import sys
import os
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')
import json
import numpy as np
from honeybee.schedule import Schedule
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as GridBasedDF
from honeybee.radiance.recipe.annual.gridbased import GridBased as GridBasedAnnual
from honeybee.futil import bat_to_sh

def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)

# Load the sky matrix
sky_mtx = load_json(r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\RADIANCE_TEST_INPUT\sky_mtx.json")

# Load the context geometry (surfaces)
surfaces = load_json(r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\RADIANCE_TEST_INPUT\surfaces.json")#["surfaces"]

# Load an analysis grid
analysis_grid = load_json(r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\RADIANCE_TEST_INPUT\analysis_grids\zone1.json")

# Annual simulaton additonal paramters
annual_config = {
    "type": "gridbased",
    "id": "annual",
    "simulation_type": 0,
    "rad_parameters": {
        "gridbased_parameters": 
        "-I -aa 0.25 -dj 0.0 -ds 0.5 -dr 3 -ss 0.0 -dp 64 -ad 5000 -st 0.85 -lw 2e-06 -as 128 -dc 0.25 -dt 0.5 -ab 3 -c 1 -lr 4 -ar 16"
    }
}

# DF based simulation
df_config = {
    "rad_parameters": {
        "gridbased_parameters": 
        "-aa 0.25 -dj 0.0 -ds 0.5 -ss 0.0 -dp 64 -ad 512 -st 0.85 -lw 0.05 -as 128 -dc 0.25 -dt 0.5 -ab 2 -lr 4 -dr 3 -ar 16"
    },
    "type": "gridbased",
    "id": "daylight_factor"
}

# Create recipe JSON origin string
DF_RECIPE_JSON = {k: v for d in [df_config, analysis_grid, surfaces] for k, v in d.items()}
ANNUAL_RECIPE_JSON = {k: v for d in [annual_config, analysis_grid, surfaces, sky_mtx] for k, v in d.items()}

# Create recipe from JSON origin string
DF_RECIPE = GridBasedDF.from_json(DF_RECIPE_JSON)
ANNUAL_RECIPE = GridBasedAnnual.from_json(ANNUAL_RECIPE_JSON)

# Generate bat files
DF_BAT_FILE = DF_RECIPE.write("RADIANCE_TEST_INPUT", str(DF_RECIPE.analysis_grids[0].name))
ANNUAL_BAT_FILE = ANNUAL_RECIPE.write("RADIANCE_TEST_INPUT", str(ANNUAL_RECIPE.analysis_grids[0].name))

# Convert bats to sh
DF_SHELL_FILE = bat_to_sh(DF_BAT_FILE)
ANNUAL_SHELL_FILE = bat_to_sh(ANNUAL_BAT_FILE)

# Run the simulation
if os.name == 'nt':
    DF_RECIPE.run(DF_BAT_FILE)
    ANNUAL_RECIPE.run(ANNUAL_BAT_FILE)
else:
    DF_RECIPE.run(DF_SHELL_FILE)
    ANNUAL_RECIPE.run(ANNUAL_SHELL_FILE)

In [ ]:
x, y, z = np.array([i["location"] for i in DF_RECIPE.results()[0].to_json()["analysis_points"]]).T
df = np.array([point["values"][0][0][6324][0] for point in DF_RECIPE.results()[0].to_json()["analysis_points"]])
da = 
cda = 
udi = 
udiless = 
udimore = 


In [ ]:
grid = ANNUAL_RECIPE.results()[0]

grid.annual_metrics()

In [ ]:
[i for i in ANNUAL_RECIPE.results()[0].points]

In [ ]:
# Post process the annual results to get daylight metrics
OCCUPANCY_SCHEDULE = Schedule.from_workday_hours(occ_hours=(8, 17), off_hours=(12, 13), weekend=(6, 7), default_value=1)

#ANNUAL_RECIPE.results()[0].annual_metrics(da_threshhold=300, udi_min_max=(100, 3000), blinds_state_ids=None, occ_schedule=OCCUPANCY_SCHEDULE)

#help(ANNUAL_RECIPE)
#ANNUAL_RECIPE#[0]["sky_mtx"]
#help(ANNUAL_RECIPE.results()[0])
#help(ANNUAL_RECIPE.results()[0])
ANNUAL_RECIPE.results()[0].result_files[0][0][0]

#DA, CDA, UDI, UDILess, UDIMore = ANNUAL_RECIPE.results()[0].annual_metrics(300, (100, 2000), None, OCCUPANCY_SCHEDULE)

In [ ]:
# Load the recipe.json
import json
import os
import re
import sys
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as GridBasedDF
from honeybee.radiance.recipe.annual.gridbased import GridBased as GridBasedAnnual
from honeybee.futil import bat_to_sh


def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)


# Load the recipe
ZONE = "CM2_S_Outer"
RECIPE_PATH = "TESTTESTTEST/zone2/annual/recipe.json"
RECIPE = load_json(RECIPE_PATH)

# Load the geometry
GEOMETRY_PATH = r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\TESTTESTTEST\geometry.json"
GEOMETRY = load_json(GEOMETRY_PATH)["surfaces"]

# Update recipe to include geometry
RECIPE["surfaces"] = GEOMETRY

# Generate the Radiance case from the reconstituted recipe
if RECIPE["id"] == "annual":
    RECIPE = GridBasedAnnual.from_json(RECIPE)
else:
    RECIPE = GridBasedDF.from_json(RECIPE)

# Generate bat file
BAT_FILE = RECIPE.write("TESTTESTTEST", "RUN_NAME")

# Convert bat to sh
SHELL_FILE = bat_to_sh(BAT_FILE)

# Fix the -dr issue
for file in [SHELL_FILE, BAT_FILE]:
    with open(file, "r") as f:
        dr_fix = re.sub("-dr (.*?) -", "-dr 3 -", f.read())
    with open(file, "w") as f:
        f.write(dr_fix)

In [ ]:
# Load an idf and modify to custom parameters
"""
Description:
    Load an IDF file and modify according to settings in a config file
Arguments:
    path [string]: JSON config file (and referenced zone_conditions_library within that config file)
Returns:
    idf file [file object]: Modified IDF file

Annotations:
    TODO - Add exposed floor and roof materials
    TODO - Fix the glazing solar heat gain assignment from the config file for different orientations
"""

import json
import sys
from eppy.modeleditor import IDF
import platform


def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)

# Load the setup configuration for this IDF modification
with open("idf_config.json", "r") as f:
    CONFIG = json.load(f)

# Load IDF ready for pre-processing and modification
IDF_FILE = CONFIG["source_idf"]
if "win" in platform.platform().lower() and "dar" not in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_windows"])
elif "linux" in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_linux"])
elif "dar" in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_os"])

EPW_FILE = CONFIG["weather_file"]
idf = IDF(IDF_FILE)

# Load the JSON file containing internal gains, schedules and setpoints
ZONE_CONDITIONS = load_json(
    CONFIG["zone_conditions_library"]
)[CONFIG["zone_template"]]

# Load the EPW file to get the location variables and store in the IDF object
with open(EPW_FILE, "r") as f:
    A, B, C, D, E, F, G, H, I, J = f.readlines()[0].replace(
        "\n", ""
    ).split(",")
idf.idfobjects["SITE:LOCATION"] = []
idf.newidfobject(
    "SITE:LOCATION",
    Name=B,
    Latitude=float(G),
    Longitude=float(H),
    Time_Zone=float(I),
    Elevation=float(J)
)

# Set version number
idf.idfobjects["VERSION"] = []
idf.newidfobject(
    "VERSION",
    Version_Identifier="8.8.0"
)

# Remove Design Day sizing periods
idf.idfobjects["SIZINGPERIOD:DESIGNDAY"] = []

# Remove surface output (to save on simulation time and results size)
idf.idfobjects["OUTPUT:SURFACES:LIST"] = []

# Remove table style output to save on results file size
idf.idfobjects["OUTPUTCONTROL:TABLE:STYLE"] = []

# Set/remove sizing parameters
idf.idfobjects["SIZING:PARAMETERS"] = []

# Remove the HVAC objects provifing fresh air from outside
idf.idfobjects["DESIGNSPECIFICATION:OUTDOORAIR"] = []

# Set simulation to run only for annual period corresponding with weatherfile
idf.idfobjects["SIMULATIONCONTROL"] = []
idf.newidfobject(
    "SIMULATIONCONTROL",
    Do_Zone_Sizing_Calculation="No",
    Do_System_Sizing_Calculation="No",
    Do_Plant_Sizing_Calculation="No",
    Run_Simulation_for_Sizing_Periods="No",
    Run_Simulation_for_Weather_File_Run_Periods="Yes"
)

# Set simulation run period (including start day of year)
idf.idfobjects["RUNPERIOD"] = []
idf.newidfobject(
    "RUNPERIOD",
    Name="Custom Run",
    Begin_Month=1,
    Begin_Day_of_Month=1,
    End_Month=12,
    End_Day_of_Month=31,
    Day_of_Week_for_Start_Day="Monday",
    Use_Weather_File_Holidays_and_Special_Days="Yes",
    Use_Weather_File_Daylight_Saving_Period="Yes",
    Apply_Weekend_Holiday_Rule="No",
    Use_Weather_File_Rain_Indicators="Yes",
    Use_Weather_File_Snow_Indicators="Yes"
)

# Remove output variable dictionary
idf.idfobjects["OUTPUT:VARIABLEDICTIONARY"] = []

# Set general building parameters (including North angle)
idf.idfobjects["BUILDING"] = []
idf.newidfobject(
    "BUILDING",
    Name="IDF Name",
    North_Axis=0,
    Terrain="City",
    Solar_Distribution="FullExteriorWithReflections",
    Maximum_Number_of_Warmup_Days=25,
    Minimum_Number_of_Warmup_Days=6
)

# Set number of timesteps per hour in simulation
idf.idfobjects["TIMESTEP"] = []
idf.newidfobject(
    "TIMESTEP",
    Number_of_Timesteps_per_Hour=6
)

# Set shadow calculation method
idf.idfobjects["SHADOWCALCULATION"] = []
idf.newidfobject(
    "SHADOWCALCULATION",
    Calculation_Method="AverageOverDaysInFrequency",
    Calculation_Frequency=20,
    Maximum_Figures_in_Shadow_Overlap_Calculations=1000
)

# Set schedule type limits
idf.idfobjects["SCHEDULETYPELIMITS"] = []
idf.newidfobject(
    "SCHEDULETYPELIMITS",
    Name="FractionLimits",
    Lower_Limit_Value=0,
    Upper_Limit_Value=1,
    Numeric_Type="Continuous",
    Unit_Type="Dimensionless")

idf.newidfobject(
    "SCHEDULETYPELIMITS",
    Name="OnOffLimits",
    Lower_Limit_Value=0,
    Upper_Limit_Value=1,
    Numeric_Type="Discrete",
    Unit_Type="Dimensionless"
)

idf.newidfobject(
    "SCHEDULETYPELIMITS",
    Name="TemperatureSetpointLimits",
    Lower_Limit_Value=0,
    Upper_Limit_Value=100,
    Numeric_Type="Continuous",
    Unit_Type="Dimensionless"
)

idf.newidfobject(
    "SCHEDULETYPELIMITS",
    Name="ActivityLevelLimits",
    Lower_Limit_Value=0,
    Upper_Limit_Value=1000,
    Numeric_Type="Continuous",
    Unit_Type="Dimensionless"
)

# Set daily profiles from the internal gains TEMPlates
idf.idfobjects["SCHEDULE:DAY:INTERVAL"] = []
idf.idfobjects["SCHEDULE:DAY:HOURLY"] = []

# Set a daily Always On profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="AlwaysOnDay",
    Schedule_Type_Limits_Name="OnOffLimits"
)

for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = 1

# Set a daily Always Off profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="AlwaysOffDay",
    Schedule_Type_Limits_Name="OnOffLimits"
)

for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = 0

# Set daily cooling profile from JSON
setpoint = ZONE_CONDITIONS["cooling_setpoint"]
setback = ZONE_CONDITIONS["cooling_setback"]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="CoolingSetpointDayWeekday",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = setpoint if ZONE_CONDITIONS[
        "cooling_setpoint_weekday"
    ]["Hour_{0:}".format(i+1)] == 0 else setback
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="CoolingSetpointDayWeekend",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = setpoint if ZONE_CONDITIONS[
        "cooling_setpoint_weekend"
    ]["Hour_{0:}".format(i+1)] == 0 else setback

# Set daily heating profile from JSON
setpoint = ZONE_CONDITIONS["heating_setpoint"]
setback = ZONE_CONDITIONS["heating_setback"]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="HeatingSetpointDayWeekday",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = setpoint if ZONE_CONDITIONS[
        "heating_setpoint_weekday"
    ]["Hour_{0:}".format(i+1)] == 0 else setback
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="HeatingSetpointDayWeekend",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = setpoint if ZONE_CONDITIONS[
        "heating_setpoint_weekend"
    ]["Hour_{0:}".format(i+1)] == 0 else setback

# Set a daily Occupant profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="OccupantGainDayWeekday",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "occupant_profile_weekday"
    ]["Hour_{0:}".format(i+1)]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="OccupantGainDayWeekend",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "occupant_profile_weekend"
    ]["Hour_{0:}".format(i+1)]

# Set a daily Lighting profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="LightingGainDayWeekday",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "lighting_profile_weekday"
    ]["Hour_{0:}".format(i+1)]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="LightingGainDayWeekend",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "lighting_profile_weekend"
    ]["Hour_{0:}".format(i+1)]

# Set a daily Equipment profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="EquipmentGainDayWeekday",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "equipment_profile_weekday"
    ]["Hour_{0:}".format(i+1)]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="EquipmentGainDayWeekend",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "equipment_profile_weekend"
    ]["Hour_{0:}".format(i+1)]

# Set a daily Ventilation profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="VentilationGainDayWeekday",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "ventilation_profile_weekday"
    ]["Hour_{0:}".format(i+1)]
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="VentilationGainDayWeekend",
    Schedule_Type_Limits_Name="FractionLimits"
)
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "ventilation_profile_weekend"
    ]["Hour_{0:}".format(i+1)]

# Set a daily occupant activity level profile
TEMP = idf.newidfobject(
    "SCHEDULE:DAY:HOURLY",
    Name="OccupantActivityLevelDay",
    Schedule_Type_Limits_Name="ActivityLevelLimits")
for i in range(24):
    TEMP["Hour_{0:}".format(i+1)] = ZONE_CONDITIONS[
        "occupant_sensible_gain_watts_per_person"
    ]+ZONE_CONDITIONS["occupant_latent_gain_watts_per_person"]

# Remove the current Weekly profiles and replace with compact weekly profiles
idf.idfobjects["SCHEDULE:WEEK:DAILY"] = []
idf.idfobjects["SCHEDULE:WEEK:COMPACT"] = []
idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="OccupantGainWeek",
    DayType_List_1="WeekDays",
    ScheduleDay_Name_1="OccupantGainDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="OccupantGainDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="LightingGainWeek",
    DayType_List_1="WeekDays",
    ScheduleDay_Name_1="LightingGainDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="LightingGainDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="EquipmentGainWeek",
    DayType_List_1="WeekDays",
    ScheduleDay_Name_1="EquipmentGainDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="EquipmentGainDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="OccupantActivityLevelWeek",
    DayType_List_1="AllDays",
    ScheduleDay_Name_1="OccupantActivityLevelDay"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="HeatingSetpointWeek",
    DayType_List_1="WeekDays",
    ScheduleDay_Name_1="HeatingSetpointDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="HeatingSetpointDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="CoolingSetpointWeek",
    DayType_List_1="WeekDays",
    ScheduleDay_Name_1="CoolingSetpointDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="CoolingSetpointDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="VentilationWeek",
    DayType_List_1="Weekdays",
    ScheduleDay_Name_1="VentilationGainDayWeekday",
    DayType_List_2="AllOtherDays",
    ScheduleDay_Name_2="VentilationGainDayWeekend"
)

idf.newidfobject(
    "SCHEDULE:WEEK:COMPACT",
    Name="AlwaysOnWeek",
    DayType_List_1="AllDays",
    ScheduleDay_Name_1="AlwaysOnDay")

# Set annual profiles
idf.idfobjects["SCHEDULE:YEAR"] = []
idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="OccupantGainYear",
    Schedule_Type_Limits_Name="FractionLimits",
    ScheduleWeek_Name_1="OccupantGainWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="LightingGainYear",
    Schedule_Type_Limits_Name="FractionLimits",
    ScheduleWeek_Name_1="LightingGainWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="EquipmentGainYear",
    Schedule_Type_Limits_Name="FractionLimits",
    ScheduleWeek_Name_1="EquipmentGainWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="OccupantActivityLevelYear",
    Schedule_Type_Limits_Name="ActivityLevelLimits",
    ScheduleWeek_Name_1="OccupantActivityLevelWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="HeatingSetpointYear",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits",
    ScheduleWeek_Name_1="HeatingSetpointWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="CoolingSetpointYear",
    Schedule_Type_Limits_Name="TemperatureSetpointLimits",
    ScheduleWeek_Name_1="CoolingSetpointWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="VentilationYear",
    Schedule_Type_Limits_Name="OnOffLimits",
    ScheduleWeek_Name_1="VentilationWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="InfiltrationYear",
    Schedule_Type_Limits_Name="OnOffLimits",
    ScheduleWeek_Name_1="AlwaysOnWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

idf.newidfobject(
    "SCHEDULE:YEAR",
    Name="Always On",
    Schedule_Type_Limits_Name="OnOffLimits",
    ScheduleWeek_Name_1="AlwaysOnWeek",
    Start_Month_1=1,
    Start_Day_1=1,
    End_Month_1=12,
    End_Day_1=31
)

# Set heating and cooling setpoints from profile loaded in TEMPlate JSON
idf.idfobjects["HVACTEMPLATE:THERMOSTAT"] = []
[idf.newidfobject(
    "HVACTEMPLATE:THERMOSTAT",
    Name=j+"_HVAC",
    Heating_Setpoint_Schedule_Name="HeatingSetpointYear",
    Constant_Heating_Setpoint="",
    Cooling_Setpoint_Schedule_Name="CoolingSetpointYear",
    Constant_Cooling_Setpoint=""
) for j in [i.Name for i in idf.idfobjects["ZONE"]]]

# Set the people gains for all spaces
idf.idfobjects["PEOPLE"] = []
idf.newidfobject(
    "PEOPLE",
    Name="PeopleGain",
    Zone_or_ZoneList_Name="AllZones",
    Number_of_People_Schedule_Name="OccupantGainYear",
    Number_of_People_Calculation_Method="Area/Person",
    Zone_Floor_Area_per_Person=ZONE_CONDITIONS["occupant_gain_m2_per_person"],
    Fraction_Radiant=0.3,
    Sensible_Heat_Fraction=float(ZONE_CONDITIONS[
        "occupant_sensible_gain_watts_per_person"
    ]) / float(sum([ZONE_CONDITIONS[
        "occupant_sensible_gain_watts_per_person"
        ], ZONE_CONDITIONS[
            "occupant_latent_gain_watts_per_person"
        ]])
    ),
    Activity_Level_Schedule_Name="OccupantActivityLevelYear"
)

# Set the lighting gains for all spaces
idf.idfobjects["LIGHTS"] = []
idf.newidfobject(
    "LIGHTS",
    Name="LightingGain",
    Zone_or_ZoneList_Name="AllZones",
    Schedule_Name="LightingGainYear",
    Design_Level_Calculation_Method="Watts/Area",
    Watts_per_Zone_Floor_Area=ZONE_CONDITIONS["lighting_gain_watts_per_m2"],
    Fraction_Radiant=0.5,
    Fraction_Visible=0.5,
    Lighting_Level=ZONE_CONDITIONS["design_illuminance_lux"]
)

# Set the equipment gains for all spaces
idf.idfobjects["ELECTRICEQUIPMENT"] = []
idf.newidfobject(
    "ELECTRICEQUIPMENT",
    Name="EquipmentGain",
    Zone_or_ZoneList_Name="AllZones",
    Schedule_Name="EquipmentGainYear",
    Design_Level_Calculation_Method="Watts/Area",
    Watts_per_Zone_Floor_Area=ZONE_CONDITIONS["equipment_gain_watts_per_m2"],
    Fraction_Radiant=0.15,
    Fraction_Latent=0.85,
    Fraction_Lost=0
)

# Set infiltration rate for all zones
idf.idfobjects["ZONEINFILTRATION:DESIGNFLOWRATE"] = []
idf.newidfobject(
    "ZONEINFILTRATION:DESIGNFLOWRATE",
    Name="InfiltrationGain",
    Zone_or_ZoneList_Name="AllZones",
    Schedule_Name="InfiltrationYear",
    Design_Flow_Rate_Calculation_Method="Flow/Area",
    Flow_per_Zone_Floor_Area=ZONE_CONDITIONS["infiltration_m3_per_second_m2"]
)

# Set ventilation rate for all zones
idf.idfobjects["ZONEVENTILATION:DESIGNFLOWRATE"] = []
idf.newidfobject(
    "ZONEVENTILATION:DESIGNFLOWRATE",
    Name="VentilationGain",
    Zone_or_ZoneList_Name="AllZones",
    Schedule_Name="VentilationYear",
    Design_Flow_Rate_Calculation_Method="Flow/Person",
    Flow_Rate_per_Person=ZONE_CONDITIONS[
        "ventilation_litres_per_second_per_person"
    ]*0.001
)

# Set Ideal Loads Air System air supply based on internal TEMPlate
idf.idfobjects["HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM"] = []
# [idf.newidfobject(
#     "HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM",
#     Zone_Name=j,
#     Template_Thermostat_Name=j+"_HVAC",
#     Maximum_Heating_Supply_Air_Temperature=50,
#     Minimum_Cooling_Supply_Air_Temperature=13,
#     Maximum_Heating_Supply_Air_Humidity_Ratio=0.0156,
#     Minimum_Cooling_Supply_Air_Humidity_Ratio=0.0077,
#     Heating_Limit="NoLimit",
#     Cooling_Limit="NoLimit",
#     Dehumidification_Control_Type="ConstantSensibleHeatRatio",
#     Cooling_Sensible_Heat_Ratio=0.7,
#     Dehumidification_Setpoint=60,
#     Humidification_Control_Type="None",
#     Humidification_Setpoint=30,
#     Outdoor_Air_Method="Flow/Person",
#     Outdoor_Air_Flow_Rate_per_Person=internal_gains_library[zone_template][
#    "ventilation_litres_per_second_per_person"
# ]*0.001,
#     Outdoor_Air_Economizer_Type="NoEconomizer",
#     Heat_Recovery_Type="None",
#     Sensible_Heat_Recovery_Effectiveness=0.7,
#     Latent_Heat_Recovery_Effectiveness=0.65) for j in [
#    i.Name for i in idf.idfobjects["ZONE"]
# ]]

# Remove the existing window materials
idf.idfobjects["WINDOWMATERIAL:GLAZING"] = []
idf.idfobjects["WINDOWMATERIAL:GAS"] = []

# Remove the existing materials
idf.idfobjects["MATERIAL:AIRGAP"] = []
idf.idfobjects["MATERIAL"] = []

# Create single layer window material for glazing transmittance/g-value
idf.idfobjects["WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM"] = []
idf.newidfobject(
    "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
    Name="EXTERIOR GLAZING MATERIAL_N",
    UFactor=CONFIG["glass_u_value"],
    Solar_Heat_Gain_Coefficient=CONFIG["glass_solar_heat_gain_coefficient_N"],
    Visible_Transmittance=CONFIG["glass_visible_transmittance_N"]
)

idf.newidfobject(
    "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
    Name="EXTERIOR GLAZING MATERIAL_E",
    UFactor=CONFIG["glass_u_value"],
    Solar_Heat_Gain_Coefficient=CONFIG["glass_solar_heat_gain_coefficient_E"],
    Visible_Transmittance=CONFIG["glass_visible_transmittance_N"]
)

idf.newidfobject(
    "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
    Name="EXTERIOR GLAZING MATERIAL_S",
    UFactor=CONFIG["glass_u_value"],
    Solar_Heat_Gain_Coefficient=CONFIG["glass_solar_heat_gain_coefficient_S"],
    Visible_Transmittance=CONFIG["glass_visible_transmittance_N"]
)

idf.newidfobject(
    "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
    Name="EXTERIOR GLAZING MATERIAL_W",
    UFactor=CONFIG["glass_u_value"],
    Solar_Heat_Gain_Coefficient=CONFIG["glass_solar_heat_gain_coefficient_W"],
    Visible_Transmittance=CONFIG["glass_visible_transmittance_N"]
)

idf.newidfobject(
    "WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM",
    Name="INTERIOR GLAZING MATERIAL",
    UFactor=0.8,
    Solar_Heat_Gain_Coefficient=0.9,
    Visible_Transmittance=0.9
)

# Create basic single layer materials with no mass for easy U-Value attribution
idf.idfobjects["MATERIAL:NOMASS"] = []
idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="EXTERIOR WALL MATERIAL",
    Roughness="MediumRough",
    Thermal_Resistance=1/CONFIG["exterior_wall_u_value"],
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=1-CONFIG["wall_reflectivity"]
)

idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="INTERIOR WALL MATERIAL",
    Roughness="MediumSmooth",
    Thermal_Resistance=1/1.8,
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=1-CONFIG["wall_reflectivity"]
)

idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="INTERIOR FLOOR MATERIAL",
    Roughness="MediumSmooth",
    Thermal_Resistance=1/1.087,
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=1-CONFIG["floor_reflectivity"]
)

idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="INTERIOR CEILING MATERIAL",
    Roughness="MediumSmooth",
    Thermal_Resistance=1/1.087,
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=1-CONFIG["ceiling_reflectivity"]
)

idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="EXTERIOR ROOF MATERIAL",
    Roughness="MediumRough",
    Thermal_Resistance=1/CONFIG["exterior_roof_u_value"],
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=1-CONFIG["ceiling_reflectivity"]
)

idf.newidfobject(
    "MATERIAL:NOMASS",
    Name="AIR WALL MATERIAL",
    Roughness="MediumRough",
    Thermal_Resistance=0.001,
    Thermal_Absorptance=0.001,
    Solar_Absorptance=0.001,
    Visible_Absorptance=0.001
)

idf.newidfobject(
    "MATERIAL",
    Name="THERMAL MASS MATERIAL",
    Roughness="MediumRough",
    Thickness=1,
    Conductivity=2,
    Density=2000,
    Specific_Heat=900,
    Thermal_Absorptance=0.9,
    Solar_Absorptance=0.7,
    Visible_Absorptance=0.7
)

# Set the constructions for the whole building
idf.idfobjects["CONSTRUCTION"] = []
idf.newidfobject(
    "CONSTRUCTION",
    Name="EXTERIOR WALL",
    Outside_Layer="EXTERIOR WALL MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="INTERIOR WALL",
    Outside_Layer="INTERIOR WALL MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="INTERIOR FLOOR",
    Outside_Layer="INTERIOR FLOOR MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="INTERIOR CEILING",
    Outside_Layer="INTERIOR CEILING MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="EXTERIOR ROOF",
    Outside_Layer="EXTERIOR ROOF MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="AIR WALL",
    Outside_Layer="AIR WALL MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="EXTERIOR WINDOW",
    Outside_Layer="EXTERIOR GLAZING MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="INTERIOR WINDOW",
    Outside_Layer="INTERIOR GLAZING MATERIAL"
)

idf.newidfobject(
    "CONSTRUCTION",
    Name="THERMAL MASS",
    Outside_Layer="THERMAL MASS MATERIAL"
)

# Get external surface areas for each zone and assign internal mass
ZONE_WALL_AREA = []
for zone in [str(i.Name) for i in idf.idfobjects["ZONE"]]:
    area = 0
    for surface in idf.idfobjects["BUILDINGSURFACE:DETAILED"]:
        if (surface.Zone_Name == zone) & (str(surface.Sun_Exposure) == "SunExposed"):
            area += surface.area
    ZONE_WALL_AREA.append(area)

idf.idfobjects["INTERNALMASS"] = []
for i, j in list(zip([str(i.Name) for i in idf.idfobjects["ZONE"]], ZONE_WALL_AREA)):
    if j != 0:
        idf.newidfobject(
            "INTERNALMASS",
            Name=i+"_MASS",
            Construction_Name="THERMAL MASS",
            Zone_Name=i,
            Surface_Area=j
        )
    else:
        pass

# Create a list zones to be referenced for passing the internal gains setpoitns
TEMP = idf.newidfobject("ZONELIST", Name="AllZones")
for i, j in enumerate([str(i.Name) for i in idf.idfobjects["ZONE"]]):
    TEMP["Zone_{0:}_Name".format(i+1)] = j

# Output variables to report during simulation
OUTPUT_VARIABLES = [
    "Zone Mean Air Temperature",
    "Zone Mean Radiant Temperature",
    "Zone Air Relative Humidity",
    "Zone Operative Temperature",
    "Zone People Total Heating Energy",
    "Zone Lights Electric Energy",
    "Zone Electric Equipment Electric Energy",
    "Zone Windows Total Transmitted Solar Radiation Energy",
    "Zone Air System Sensible Heating Energy",
    "Zone Air System Sensible Cooling Energy",
]

# Set the list of outputs to be generated fromt eh EnergyPLus simulation
idf.idfobjects["OUTPUT:VARIABLE"] = []
[
    idf.newidfobject(
        "OUTPUT:VARIABLE",
        Key_Value="*",
        Variable_Name=i,
        Reporting_Frequency="hourly"
    ) for i in OUTPUT_VARIABLES
]

# Diagnostics/testing
"""
The following section adds a bunch of diagnostics toools that can be used for checking how well the simulation has run
This includes:
    A list of potential output variables possible from the completed/failed simulation
    A list of the constructions in the completed/failed simulation - including U-values and thermal mass
    An SQLite format output result from the completed/failed simulation - this is probably better than using ReadVarsESO, but needs some further work]
    A detailed output diagnostics file indicatign any major issues in the completed/failed simulation
"""
# idf.idfobjects["OUTPUT:VARIABLEDICTIONARY"] = []
# idf.newidfobject("OUTPUT:VARIABLEDICTIONARY", Key_Field="regular")

# idf.idfobjects["OUTPUT:CONSTRUCTIONS"] = []
# idf.newidfobject("OUTPUT:CONSTRUCTIONS", Details_Type_1="Constructions")

# idf.idfobjects["OUTPUT:SQLITE"] = []
# idf.newidfobject("OUTPUT:SQLITE", Option_Type="Simple")

# idf.idfobjects["OUTPUT:DIAGNOSTICS"] = []
# idf.newidfobject("OUTPUT:DIAGNOSTICS", Key_1="DisplayExtraWarnings", Key_2="DisplayUnusedSchedules")

# Save the idf to a new file
idf.saveas(CONFIG["target_idf"])

In [ ]:

top = []
for parent in [[i.Name, i.coords] for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"]]:
    mid = []
    for child in [[i.Name, i.coords] for i in idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]]:
        if parent[0] in child[0]:
            mid.append(child[1])
    top.append([parent[1], mid])
top[9]

In [ ]:
# Create Radiance recipes from IDF
"""
Description:
    Load an IDF file and generate the input/s for a Radiance simulation
Arguments:
    path [string]: JSON config file (and referenced IDF within that config file)
Returns:
    Radiance simulation recipe/s [file objects]: Radiance simulation input/s

Annotations:
    TODO - Orient analysis grid to longest edge of zone it's being generated for
    TODO - Create ability to visualise surfaces and analsyis grids in context
    TODO - Add radiance parameters to the config.json to put into the generated recipes
    TODO - Add method interpreting results for SDA, DA, DF, UDI, UDILess, UDIMore
    TODO - Replace sys.stdout with print() statements giving indication of current running process
"""

# Load necessary packages
import sys
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')
from honeybee.hbsurface import HBSurface
from honeybee.hbfensurface import HBFenSurface
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.properties import RadianceProperties
from honeybee.radiance.material.glass import Glass
from honeybee.radiance.material.plastic import Plastic
import matplotlib.patches as patches
from honeybee.radiance.sky.skymatrix import SkyMatrix
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as GridBasedDF
from honeybee.radiance.recipe.annual.gridbased import GridBased as GridBasedAnnual
import os
import eppy
from eppy.modeleditor import IDF
import json
import numpy as np
import platform


def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)

# Specify the config file to be referenced
CONFIG = load_json("idf_config.json")
print("\n{0:} loaded\n".format("idf_config.json"))

# Load IDF ready for pre-processing and radaince case preparation
IDF_FILE = CONFIG["source_idf"]
if "win" in platform.platform().lower() and "dar" not in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_windows"])
elif "linux" in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_linux"])
elif "dar" in platform.platform().lower():
    IDF.setiddname(CONFIG["idd_file_os"])

EPW_FILE = CONFIG["weather_file"]
print("{0:} loaded\n".format(CONFIG["weather_file"]))

idf = IDF(IDF_FILE)
print("{0:} loaded\n".format(IDF_FILE))

# Set the "vector to north", so that wall orientation can be obtained
north_angle = np.radians(idf.idfobjects["BUILDING"][0].North_Axis)
north_vector = (np.sin(north_angle), np.cos(north_angle), 0)
print("North angle has been read as {0:}\n".format(north_angle))


def unit_vector(vector):
    """
    Description:
        Returns the unit vector of a vector
    Arguments:
        vector [1D-array]: Vector defined as [n.n, n.n, n.n]
    Returns:
        vector [1D-array]: Dictionary containing contents of loaded JSON file
    """
    return vector / np.linalg.norm(vector)


def angle_between(vector_1, vector_2):
    """ Returns the angle in radians between vectors 'vector_1' and 'vector_2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    vector_1_u = unit_vector(vector_1)
    vector_2_u = unit_vector(vector_2)
    return np.arccos(np.clip(np.dot(vector_1_u, vector_2_u), -1.0, 1.0))


def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    o_x, o_y = origin
    p_x, p_y = point

    q_x = o_x + np.cos(angle) * (p_x - o_x) - np.sin(angle) * (p_y - o_y)
    q_y = o_y + np.sin(angle) * (p_x - o_x) + np.cos(angle) * (p_y - o_y)
    return q_x, q_y

# Define materials to be applied to surfaces
glass_material_N = Glass(
    "GlassMaterialN",
    r_transmittance=CONFIG["glass_visible_transmittance_N"],
    g_transmittance=CONFIG["glass_visible_transmittance_N"],
    b_transmittance=CONFIG["glass_visible_transmittance_N"],
    refraction_index=1.52
    )

glass_material_S = Glass(
    "GlassMaterialS",
    r_transmittance=CONFIG["glass_visible_transmittance_S"],
    g_transmittance=CONFIG["glass_visible_transmittance_S"],
    b_transmittance=CONFIG["glass_visible_transmittance_S"],
    refraction_index=1.52
)

glass_material_E = Glass(
    "GlassMaterialE",
    r_transmittance=CONFIG["glass_visible_transmittance_E"],
    g_transmittance=CONFIG["glass_visible_transmittance_E"],
    b_transmittance=CONFIG["glass_visible_transmittance_E"],
    refraction_index=1.52
)

glass_material_W = Glass(
    "GlassMaterialW",
    r_transmittance=CONFIG["glass_visible_transmittance_W"],
    g_transmittance=CONFIG["glass_visible_transmittance_W"],
    b_transmittance=CONFIG["glass_visible_transmittance_W"],
    refraction_index=1.52
)

glass_material_interior = Glass(
    "GlassMaterialInternal",
    r_transmittance=0.9,
    g_transmittance=0.9,
    b_transmittance=0.9,
    refraction_index=1.52
)

glass_material_skylight = Glass(
    "GlassMaterialSkylight",
    r_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    g_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    b_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    refraction_index=1.52
)

air_wall_material = Glass(
    "AirWallMaterial",
    r_transmittance=0,
    g_transmittance=0,
    b_transmittance=0,
    refraction_index=1
)

wall_material = Plastic(
    "WallMaterial",
    r_reflectance=CONFIG["wall_reflectivity"],
    g_reflectance=CONFIG["wall_reflectivity"],
    b_reflectance=CONFIG["wall_reflectivity"],
    specularity=0,
    roughness=0
)

ceiling_material = Plastic(
    "CeilingMaterial",
    r_reflectance=CONFIG["ceiling_reflectivity"],
    g_reflectance=CONFIG["ceiling_reflectivity"],
    b_reflectance=CONFIG["ceiling_reflectivity"],
    specularity=0,
    roughness=0
)

floor_material = Plastic(
    "FloorMaterial",
    r_reflectance=CONFIG["floor_reflectivity"],
    g_reflectance=CONFIG["floor_reflectivity"],
    b_reflectance=CONFIG["floor_reflectivity"],
    specularity=0,
    roughness=0
)

print("Materials defined from reflectivity and transmissivity in {0:}\n".format("idf_config.json"))

# Assign surfaces within IDF to HBSurfaces for daylight analysis
print("Generating surfaces:")
EXTERIOR_WALL_SURFACES = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Exterior Wall"]):
    srf = HBSurface(
        "wall_{0:}_{1:}".format(wall_n, wall.Name),
        wall.coords,
        surface_type=0,
        is_name_set_by_user=True,
        is_type_set_by_user=True,
        rad_properties=RadianceProperties(
            material=wall_material
        )
    )
    angle_to_north = np.degrees(angle_between(north_vector, srf.normal))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            # Assign glass properties based on orientation
            if (angle_to_north > 315) or (angle_to_north <= 45):
                orientation_glass_material = glass_material_N
            elif (angle_to_north > 45) and (angle_to_north <= 135):
                orientation_glass_material = glass_material_E
            elif (angle_to_north > 135) and (angle_to_north <= 225):
                orientation_glass_material = glass_material_S
            elif (angle_to_north > 225) and (angle_to_north <= 315):
                orientation_glass_material = glass_material_W
            fensrf = HBFenSurface(
                "wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name),
                fenestration.coords,
                rad_properties=RadianceProperties(
                    material=orientation_glass_material
                )
            )
            srf.add_fenestration_surface(fensrf)
    EXTERIOR_WALL_SURFACES.append(srf)

print("{0:} exterior wall surfaces generated".format(len(EXTERIOR_WALL_SURFACES)))

INTERIOR_WALL_SURFACES = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Interior Wall"]):
    srf = HBSurface(
        "wall_{0:}_{1:}".format(wall_n, wall.Name),
        wall.coords,
        surface_type=0,
        is_name_set_by_user=True,
        is_type_set_by_user=True,
        rad_properties=RadianceProperties(
            material=wall_material
        )
    )
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            fensrf = HBFenSurface(
                "wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name),
                fenestration.coords,
                rad_properties=RadianceProperties(
                    material=glass_material_interior
                )
            )
            srf.add_fenestration_surface(fensrf)
    INTERIOR_WALL_SURFACES.append(srf)

print("{0:} interior wall surfaces generated".format(len(INTERIOR_WALL_SURFACES)))

AIRWALL_SURFACES = []
for airwall_n, airwall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Air Wall"]):
    srf = HBSurface("airwall_{0:}_{1:}".format(airwall_n, airwall.Name), airwall.coords, surface_type=4, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=air_wall_material))
    AIRWALL_SURFACES.append(srf)

print("{0:} air wall surfaces generated".format(len(AIRWALL_SURFACES)))

FLOOR_SURFACES = []
for floor_n, floor in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Floor") or (i.Construction_Name == "Exterior Floor") or (i.Construction_Name == "Exposed Floor")]):
    srf = HBSurface("floor_{0:}_{1:}".format(floor_n, floor.Name), floor.coords, surface_type=2, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=floor_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if floor.Name in fenestration.Name:
            fensrf = HBFenSurface("floor_{0:}_{1:}_fenestration_{2:}_{3:}".format(floor_n, floor.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_interior))
            srf.add_fenestration_surface(fensrf)
    FLOOR_SURFACES.append(srf)

print("{0:} floor surfaces generated".format(len(FLOOR_SURFACES)))

CEILING_SURFACES = []
for ceiling_n, ceiling in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Ceiling") or (i.Construction_Name == "Exterior Ceiling") or (i.Construction_Name == "Roof")]):
    srf = HBSurface("ceiling_{0:}_{1:}".format(ceiling_n, ceiling.Name), ceiling.coords, surface_type=3, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=ceiling_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if ceiling.Name in fenestration.Name:
            fensrf = HBFenSurface("ceiling_{0:}_{1:}_fenestration_{2:}_{3:}".format(ceiling_n, ceiling.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_skylight))
            srf.add_fenestration_surface(fensrf)
    CEILING_SURFACES.append(srf)

print("{0:} ceiling surfaces generated".format(len(CEILING_SURFACES)))

CONTEXT_SURFACES = []
for context_n, context in enumerate([i for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]):
    srf = HBSurface("context_{0:}_{1:}".format(context_n, context.Name), context.coords, surface_type=6, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    CONTEXT_SURFACES.append(srf)

print("{0:} shading surfaces generated\n".format(len(CONTEXT_SURFACES)))

HB_OBJECTS = np.concatenate([EXTERIOR_WALL_SURFACES, INTERIOR_WALL_SURFACES, FLOOR_SURFACES, CEILING_SURFACES, AIRWALL_SURFACES, CONTEXT_SURFACES]).tolist()

# Define analysis grids for each zone for simulation in Radiance
print("Generating analysis grids: ")
HB_ANALYSIS_GRIDS = []
for floor_srf in [i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if ("Floor" in i.Construction_Name)]:
    vert_xs, vert_ys, vert_zs = list(zip(*floor_srf.coords))
    patch = patches.Polygon(list(zip(*[vert_xs, vert_ys])))
    min_x, max_x, min_y, max_y, max_z = min(vert_xs), max(vert_xs), min(vert_ys), max(vert_ys), max(vert_zs)
    x_range = max_x - min_x
    y_range = max_y - min_y
    g = np.meshgrid(
        np.arange(min_x - (x_range / 2), max_x + (x_range / 2), CONFIG["daylight_analysis_grid_spacing"]),
        np.arange(min_y - (y_range / 2), max_y + (y_range / 2), CONFIG["daylight_analysis_grid_spacing"])
    )
    COORDS = list(zip(*(c.flat for c in g)))
    ANALYSIS_POINTS = np.vstack([p for p in COORDS if patch.contains_point(p, radius=CONFIG["daylight_analysis_grid_edge_offset"])])
    GRID_POINTS = list(zip(*[np.array(list(zip(*ANALYSIS_POINTS)))[0], np.array(list(zip(*ANALYSIS_POINTS)))[1], np.repeat(max_z+CONFIG["daylight_analysis_grid_surface_offset"], len(ANALYSIS_POINTS))]))
    HB_ANALYSIS_GRIDS.append(AnalysisGrid.from_points_and_vectors(GRID_POINTS, name=floor_srf.Zone_Name))
    print("Analysis grid for {0:} generated ({1:} points)".format(floor_srf.Zone_Name, len(ANALYSIS_POINTS)))

# Generate sky matrix for annual analysis
print("\nGenerating sky matrix ...")
SKY_MATRIX = SkyMatrix.from_epw_file(EPW_FILE, sky_density=2, north=0, hoys=range(0, 8760), mode=0, suffix="")
print("Sky matrix ({0:}) generated\n".format(SKY_MATRIX))

# Create the analysis recipes for each IDF zone
print("Generating analysis grids ...")
for HB_ANALYSIS_GRID in HB_ANALYSIS_GRIDS:

    # Create a directory in which to save the DF recipe/s
    DF_RECIPE_DIR = "{0:}/{1:}/daylight_factor".format(CONFIG["output_directory"], HB_ANALYSIS_GRID.name)
    if not os.path.exists(DF_RECIPE_DIR):
        os.makedirs(DF_RECIPE_DIR)

    # Generate a DF recipe as JSON and save [WITHOUT CONTEXT GEOMETRY]
    with open("{0:}/recipe.json".format(DF_RECIPE_DIR), "w") as f:
        json.dump(GridBasedDF(analysis_grids=[HB_ANALYSIS_GRID], hb_objects=[]).to_json(), f)
    print("{0:} daylight factor analysis grid written to {1:}/recipe.json\n".format(HB_ANALYSIS_GRID.name, DF_RECIPE_DIR))

    # Create a directory in which to save the Annual recipe/s
    ANNUAL_RECIPE_DIR = "{0:}/{1:}/annual".format(CONFIG["output_directory"], HB_ANALYSIS_GRID.name)
    if not os.path.exists(ANNUAL_RECIPE_DIR):
        os.makedirs(ANNUAL_RECIPE_DIR)

    # Generate an ANNUAL recipe as JSON and save [WITHOUT CONTEXT GEOMETRY]
    with open("{0:}/recipe.json".format(ANNUAL_RECIPE_DIR), "w") as f:
        json.dump(GridBasedAnnual(SKY_MATRIX, analysis_grids=[HB_ANALYSIS_GRID], hb_objects=[]).to_json(), f)
    print("{0:} annual analysis grid written to {1:}/recipe.json\n".format(HB_ANALYSIS_GRID.name, ANNUAL_RECIPE_DIR))

# Write the context geometry to a seperate file
with open(CONFIG["output_directory"]+"/geometry.json", "w") as f:
    f.write(repr({"surfaces": [i.to_json() for i in HB_OBJECTS]}).replace("'", '"').replace("(", '[').replace(")", ']'))
print("Geometry written to {0:}\n".format(CONFIG["output_directory"]+"/geometry.json"))


In [ ]:
%reset -f

In [ ]:
# Load the recipe.json
import json
import os
import re
import sys
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as GridBasedDF
from honeybee.radiance.recipe.annual.gridbased import GridBased as GridBasedAnnual
from honeybee.futil import bat_to_sh


def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)


# Load the recipe
ZONE = "CM2_S_Outer"
RECIPE_PATH = "TESTTESTTEST/zone2/annual/recipe.json"
RECIPE = load_json(RECIPE_PATH)

# Load the geometry
GEOMETRY_PATH = r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\TESTTESTTEST\geometry.json"
GEOMETRY = load_json(GEOMETRY_PATH)["surfaces"]

# Update recipe to include geometry
RECIPE["surfaces"] = GEOMETRY

# Generate the Radiance case from the reconstituted recipe
if RECIPE["id"] == "annual":
    RECIPE = GridBasedAnnual.from_json(RECIPE)
else:
    RECIPE = GridBasedDF.from_json(RECIPE)

# Generate bat file
BAT_FILE = RECIPE.write("TESTTESTTEST", "RUN_NAME")

# Convert bat to sh
SHELL_FILE = bat_to_sh(BAT_FILE)

# Fix the -dr issue
for file in [SHELL_FILE, BAT_FILE]:
    with open(file, "r") as f:
        dr_fix = re.sub("-dr (.*?) -", "-dr 3 -", f.read())
    with open(file, "w") as f:
        f.write(dr_fix)

In [ ]:
"C:\EnergyPlusV8-8-0\energyplus.exe" -a -x -r -w "GBR_Cardiff_CIBSE_TRY.epw" "test_modified.idf"
"/Applications/EnergyPlus-8-8-0/energyplus-8.8.0" -a -x -r -w "GBR_Cardiff_CIBSE_TRY.epw" "test_basic_modified.idf"

In [ ]:
# Assign surfaces within IDF to HBSurfaces for daylight analysis
print("Generating surfaces:")
EXTERIOR_WALL_SURFACES = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Exterior Wall"]):
    srf = HBSurface(
        "wall_{0:}_{1:}".format(wall_n, wall.Name),
        wall.coords,
        surface_type=0,
        is_name_set_by_user=True,
        is_type_set_by_user=True,
        rad_properties=RadianceProperties(
            material=wall_material
        )
    )
    angle_to_north = np.degrees(angle_between(north_vector, srf.normal))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            # Assign glass properties based on orientation
            if (angle_to_north > 315) or (angle_to_north <= 45):
                orientation_glass_material = glass_material_N
            elif (angle_to_north > 45) and (angle_to_north <= 135):
                orientation_glass_material = glass_material_E
            elif (angle_to_north > 135) and (angle_to_north <= 225):
                orientation_glass_material = glass_material_S
            elif (angle_to_north > 225) and (angle_to_north <= 315):
                orientation_glass_material = glass_material_W
            fensrf = HBFenSurface(
                "wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name),
                fenestration.coords,
                rad_properties=RadianceProperties(
                    material=orientation_glass_material
                )
            )
            srf.add_fenestration_surface(fensrf)
    EXTERIOR_WALL_SURFACES.append(srf)

print("{0:} exterior wall surfaces generated".format(len(EXTERIOR_WALL_SURFACES)))

INTERIOR_WALL_SURFACES = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Interior Wall"]):
    srf = HBSurface(
        "wall_{0:}_{1:}".format(wall_n, wall.Name),
        wall.coords,
        surface_type=0,
        is_name_set_by_user=True,
        is_type_set_by_user=True,
        rad_properties=RadianceProperties(
            material=wall_material
        )
    )
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            fensrf = HBFenSurface(
                "wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name),
                fenestration.coords,
                rad_properties=RadianceProperties(
                    material=glass_material_interior
                )
            )
            srf.add_fenestration_surface(fensrf)
    INTERIOR_WALL_SURFACES.append(srf)

print("{0:} interior wall surfaces generated".format(len(INTERIOR_WALL_SURFACES)))

# AIRWALL_SURFACES = []
# for airwall_n, airwall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Air Wall"]):
#     srf = HBSurface("airwall_{0:}_{1:}".format(airwall_n, airwall.Name), airwall.coords, surface_type=4, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=air_wall_material))
#     AIRWALL_SURFACES.append(srf)

# print("{0:} air wall surfaces generated".format(len(AIRWALL_SURFACES)))

FLOOR_SURFACES = []
for floor_n, floor in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Floor") or (i.Construction_Name == "Exterior Floor") or (i.Construction_Name == "Exposed Floor")]):
    srf = HBSurface("floor_{0:}_{1:}".format(floor_n, floor.Name), floor.coords, surface_type=2, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=floor_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if floor.Name in fenestration.Name:
            fensrf = HBFenSurface("floor_{0:}_{1:}_fenestration_{2:}_{3:}".format(floor_n, floor.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_interior))
            srf.add_fenestration_surface(fensrf)
    FLOOR_SURFACES.append(srf)

print("{0:} floor surfaces generated".format(len(FLOOR_SURFACES)))

CEILING_SURFACES = []
for ceiling_n, ceiling in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Ceiling") or (i.Construction_Name == "Exterior Ceiling") or (i.Construction_Name == "Roof")]):
    srf = HBSurface("ceiling_{0:}_{1:}".format(ceiling_n, ceiling.Name), ceiling.coords, surface_type=3, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=ceiling_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if ceiling.Name in fenestration.Name:
            fensrf = HBFenSurface("ceiling_{0:}_{1:}_fenestration_{2:}_{3:}".format(ceiling_n, ceiling.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_skylight))
            srf.add_fenestration_surface(fensrf)
    CEILING_SURFACES.append(srf)

print("{0:} ceiling surfaces generated".format(len(CEILING_SURFACES)))

CONTEXT_SURFACES = []
for context_n, context in enumerate([i for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]):
    srf = HBSurface("context_{0:}_{1:}".format(context_n, context.Name), context.coords, surface_type=6, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    CONTEXT_SURFACES.append(srf)

print("{0:} shading surfaces generated\n".format(len(CONTEXT_SURFACES)))

HB_OBJECTS = np.concatenate([EXTERIOR_WALL_SURFACES, INTERIOR_WALL_SURFACES, FLOOR_SURFACES, CEILING_SURFACES, AIRWALL_SURFACES, CONTEXT_SURFACES]).tolist()


